In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

## Data Collection for Netflix, AmazonPrime,Disney+ and Hulu Movies from [justwatch.com](https://www.justwatch.com)

Web Scraping movies from streaming platfroms(Netflix,Amazon-Prime, Disney+ and Hulu) from a website 'justwatch.com'using selenium. The Movies scraped along with other information was about 1500+ and this would be used for further analysis

In [5]:
#create a function to scroll the webpage using selenium

def scroll_down(driver):
    # Scroll down to load more movies
    for i in range(50):  # adjust range based on how many times you want to scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(8)
        try:
            driver.find_element(By.CSS_SELECTOR, ".custom-infinite-scroll.custom-infinite-scroll--hidden-spinner").click()
        except:
            break

    SCROLL_PAUSE_TIME = 10
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height 

## Getting all  Movies Data From 2013-2022

In [6]:
#create a Function to gather all the urls as well as the info needed(genres,titles,year,runtime etc)

def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()

    # Send a request to the website
    url = 'https://www.justwatch.com/us/provider/amazon-prime-video/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

# Create empty lists for movie data
urls = get_movies_data()
Movies = []
Released_year = []
IMDB_Ratings = []
Genres = []
Age_Rating = []
Runtimes = []
Directors = []
# Streaming_platforms = []

# for url in urls:
#     # Get the movie title
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     title_block = soup.find('div', {'class': 'title-block'})
#     title = title_block.find('h1').text.strip()
#     Movies.append(title)
#     print(f"Title: {title}")

#     # Get the released year
#     year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
#     year = year_div.text.strip()[1:5] if year_div else 'N/A'
#     Released_year.append(year)
#     print(f"Released year: {year}")

#     # Get the IMDb rating
#     def get_imdb_rating(url):
#         response = requests.get(url)
#         soup = BeautifulSoup(response.content, 'html.parser')
#         try:
#             imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
#             rating = imdb_tag.text.strip().split()[0]
#         except AttributeError:
#             rating = 'N/A'
#         return rating
#     rating = get_imdb_rating(url)
#     ratings_list.append(rating)
#     print(f"IMDb rating: {rating}")

#     # Get the TV age rating
#     age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
#     if age_rating_div:
#         age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
#     else:
#         age_rating = 'N/A'
#     Age_Rating.append(age_rating)
#     print(f"Age rating: {age_rating}")
    
#     #get the movie genres
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, 'html.parser')
#     genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
#     try:
#         genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
#     except AttributeError:
#         genre = 'N/A'
#     genres.append(genre)
#     print(f'Genres: {genre}')

#     # Get the movie runtime
#     runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
#     runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
#     runtimes.append(runtime)
#     print(f"Runtime: {runtime}")

#     # Get the movie director
#     director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
#     if director_div:
#         director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip()
#         directors.append(director_name)
#     else:
#         directors.append('N/A')
#     print(f"Director: {director_name}\n")




def scrape_movie_data(url):
    # Function to scrape movie data from a given URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
    age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip() if age_rating_div else 'N/A'
    genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
    genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip() if genre_div else 'N/A'
    runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
    runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
    director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip() if director_div else 'N/A'
    imdb_rating = get_imdb_rating(url)
    streaming_platform_img = soup.find('img', {'class': 'offer__icon'})
    streaming_platform = streaming_platform_img['alt'] if streaming_platform_img else 'N/A'

    return {
        "Title": title,
        "Released Year": year,
        "Age Rating": age_rating,
        "Genres": genre,
        "Runtime": runtime,
        "Director": director_name,
        "IMDb Rating": imdb_rating,
        "Streaming Platform": streaming_platform
    }


NameError: name 'webdriver' is not defined

In [ ]:
# movie_data = {
#     'title': Movies,
#     'year': Released_year,
#     'genre': Genres,
#     'runtime': Runtimes,
#     'rating': IMDB_Ratings,
#     'age_rating': Age_Rating,
#     'director_name':Directors
# }

movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': Genres,
    'runtime': Runtimes,
    'rating': IMDB_Ratings,
    'age_rating': Age_Rating,
    'director_name':Directors,
    'streaming_platform': Streaming_platforms
}

data_df = pd.DataFrame(movie_data)


# urls = get_movies_data()
# Movies = []
# Released_year = []
# IMDB_Ratings = []
# Genres = []
# Age_Rating = []
# Runtimes = []
# Directors = []
# Streaming_platforms = []


In [7]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# Function to scroll down and load more movies
def scroll_down(driver):
    for i in range(50):  # adjust range based on how many times you want to scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(8)
        try:
            driver.find_element(By.CSS_SELECTOR, ".custom-infinite-scroll.custom-infinite-scroll--hidden-spinner").click()
        except:
            pass

def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()

    # Send a request to the website
    url = 'https://www.justwatch.com/us/provider/netflix/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

def extract_movie_titles_and_imdb_urls():
    # Create empty lists for movie titles and IMDb rating URLs
    titles = []
    imdb_urls = []

    # Get the list of movie URLs
    urls = get_movies_data()

    for url in urls:
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract movie title
        title_block = soup.find('div', {'class': 'title-block'})
        title = title_block.find('h1').text.strip()
        titles.append(title)

        # Extract IMDb rating URL
#         imdb_url = soup.find('a', {'class': '', 'href': True})
#         imdb_url = imdb_url['href'] if imdb_url and imdb_url.find('img', alt='IMDB') else 'N/A'
#         imdb_urls.append(imdb_url)



# Search for IMDb rating within the HTML content
          imdb_rating_element = soup.find('img', alt='IMDB')
    

if imdb_rating_element:
    # Extract the IMDb rating from the parent span
    imdb_rating = imdb_rating_element.find_parent('span').text.strip()
    print(f"IMDb Rating: {imdb_rating}")
else:
    print("IMDb rating element not found")

          

    return titles, imdb_urls

# Example usage:
movie_titles, imdb_rating_urls = extract_movie_titles_and_imdb_urls()

# Print the movie titles and IMDb rating URLs
for title, imdb_url in zip(movie_titles, imdb_rating_urls):
    print(f"Movie Title: {title}")
    print(f"IMDb Rating URL: {imdb_url}")
    print()



AttributeError: 'NoneType' object has no attribute 'find'

In [4]:
url = 'https://www.justwatch.com/us/movie/jigsaw-2017'
driver = webdriver.Chrome()
driver.get(url)

    # Scroll down to load more movies
scroll_down(driver)

html = driver.page_source
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
title_block = soup.find('div', {'class': 'title-block'})
title = title_block.find('h1').text.strip()
year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
year = year_div.text.strip()[1:5] if year_div else 'N/A'
age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip() if age_rating_div else 'N/A'
genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip() if genre_div else 'N/A'
# runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
# runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip() if director_div else 'N/A'
imdb_rating = get_imdb_rating(url)
streaming_platform_img = soup.find('img', {'class': 'offer__icon'})
streaming_platform = streaming_platform_img['alt'] if streaming_platform_img else 'N/A'
# Get the HTML content of the website
                

NameError: name 'webdriver' is not defined

In [ ]:
# Return a dictionary with all the movie data




movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': genres,
    'runtime': runtimes,
    'rating': ratings_list,
    'age_rating': Age_Rating,
    'director_name':directors
}


Netflix_df = pd.DataFrame(movie_data)

In [ ]:
len(urls)

In [ ]:
a = get_movies_data()

In [ ]:
# save the combined DataFrame as a CSV file
Netflix_df.to_csv('Netflix_df.csv', index=False)

 ## Getting all AmazonPrime Movies Data From 2013-2022.

In [ ]:
def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()

    # Send a request to the website
    url = 'https://www.justwatch.com/us/provider/amazon-prime-video/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

# Create empty lists for movie data
urls = get_movies_data()
Movies = []
Released_year = []
ratings_list = []
genres = []
Age_Rating = []
runtimes = []
directors = []

for url in urls:
    # Get the movie title
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    Movies.append(title)
    print(f"Title: {title}")

    # Get the released year
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    Released_year.append(year)
    print(f"Released year: {year}")

#     # Get the IMDb rating
#     def get_imdb_rating(url):
#         response = requests.get(url)
#         soup = BeautifulSoup(response.content, 'html.parser')
#         try:
#             imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
#             rating = imdb_tag.text.strip().split()[0]
#         except AttributeError:
#             rating = 'N/A'
#         return rating
#     rating = get_imdb_rating(url)
#     ratings_list.append(rating)
#     print(f"IMDb rating: {rating}")

#     # Get the TV age rating
#     age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
#     if age_rating_div:
#         age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
#     else:
#         age_rating = 'N/A'
#     Age_Rating.append(age_rating)
#     print(f"Age rating: {age_rating}")
    
#     #get the movie genres
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, 'html.parser')
#     genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
#     try:
#         genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
#     except AttributeError:
#         genre = 'N/A'
#     genres.append(genre)
#     print(f'Genres: {genre}')

#     # Get the movie runtime
#     runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
#     runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
#     runtimes.append(runtime)
#     print(f"Runtime: {runtime}")

#     # Get the movie director
#     director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
#     if director_div:
#         director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip()
#         directors.append(director_name)
#     else:
#         directors.append('N/A')
#     print(f"Director: {director_name}\n")



In [ ]:
len(urls)

In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# URL of the site
url = 'https://www.justwatch.com/us/movie/violent-night'

# Initialize a Selenium WebDriver (make sure you have the WebDriver installed and in your PATH)
driver = webdriver.Chrome()

# Open the webpage
driver.get(url)

# Wait for the page to load (adjust the time based on your internet speed)
driver.implicitly_wait(10)

# Get the HTML content after the page has fully loaded
html = driver.page_source

# Close the WebDriver as we have the HTML content
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the IMDb rating
rating_div = soup.find('div', class_='jw-scoring-listing__rating')
imdb_rating = rating_div.find('span').text.strip()

# Extract the IMDb rating value
imdb_rating_value = imdb_rating.split()[0]

# Extract the title poster URL
imgs = soup.find_all('img', {'class': 'picture-comp__img'})
title_poster_url = None
for img in imgs:
    if 'alt' in img.attrs and img['alt'] == 'Poster':
        title_poster_url = img['src']
        break

# Print the results
print('IMDb Rating:', imdb_rating_value)
print('Title Poster URL:', title_poster_url)


IMDb Rating: 80%
Title Poster URL: None


In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver

# URL of the site
url = 'https://www.justwatch.com/us/movie/violent-night'

# Initialize a Selenium WebDriver (make sure you have the WebDriver installed and in your PATH)
driver = webdriver.Chrome()

# Open the webpage
driver.get(url)

# Wait for the page to load (adjust the time based on your internet speed)
driver.implicitly_wait(10)

# Get the HTML content after the page has fully loaded
html = driver.page_source

# Close the WebDriver as we have the HTML content
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the IMDb rating
rating_div = soup.find('div', class_='jw-scoring-listing__rating')
imdb_rating = rating_div.find('span').text.strip()

# Extract the IMDb rating value
imdb_rating_value = imdb_rating.split()[0]

# Extract the title poster URL
imgs = soup.find_all('img', {'class': 'picture-comp__img'})
title_poster_url = None
for img in imgs:
    if 'alt' in img.attrs and img['alt'] == 'Poster':
        title_poster_url = img['src']
        break

# Extract the IMDb URL based on the provided code
imdb_url_tag = soup.find('a', {'class': '', 'href': True})
imdb_url = imdb_url_tag['href'] if imdb_url_tag and imdb_url_tag.find('img', alt='IMDB') else None

# Print the results
print('IMDb Rating:', imdb_rating_value)
print('Title Poster URL:', title_poster_url)
print('IMDb URL:', imdb_url)


IMDb Rating: 80%
Title Poster URL: None
IMDb URL: None


In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver

# Function to extract IMDb rating from the provided HTML tags
def extract_imdb_rating(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    imdb_rating_span = soup.find('span', {'tooltip': 'IMDB', 'class': 'lazyloaded'})
    if imdb_rating_span:
        imdb_rating = imdb_rating_span.text.strip()
        return imdb_rating

# URL of the site
url = 'https://www.justwatch.com/us/movie/violent-night'

# Set up Selenium WebDriver
driver = webdriver.Chrome()
driver.get(url)

# Execute JavaScript to get title poster image URL
poster_img_url = driver.execute_script('return document.querySelector(".picture-comp__img").src')

# Get HTML content of the page
html_content = driver.page_source

# Extract IMDb rating using BeautifulSoup
imdb_rating = extract_imdb_rating(html_content)

# Close the Selenium WebDriver
driver.quit()

# Print results
print(f'Title Poster Image URL: {poster_img_url}')
print(f'IMDb Rating: {imdb_rating}')



Title Poster Image URL: https://www.justwatch.com/images/backdrop/302258335/s1440/violent-night/violent-night
IMDb Rating: None


In [12]:
from bs4 import BeautifulSoup
from selenium import webdriver

# Function to extract IMDb rating from the provided HTML tags
def extract_imdb_rating(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    imdb_rating_span = soup.find('span', {'tooltip': 'IMDB', 'class': 'lazyloaded'})
    if imdb_rating_span:
        imdb_rating = imdb_rating_span.find('span').text.strip()
        return imdb_rating

# URL of the site
url = 'https://www.justwatch.com/us/movie/violent-night'

# Set up Selenium WebDriver
driver = webdriver.Chrome()
driver.get(url)

# Execute JavaScript to get title poster image URL
poster_img_url = driver.execute_script('return document.querySelector(".picture-comp__img").src')

# Get HTML content of the page
html_content = driver.page_source

# Extract IMDb rating using BeautifulSoup
imdb_rating = extract_imdb_rating(html_content)

# Close the Selenium WebDriver
driver.quit()

# Print results
print(f'Title Poster Image URL: {poster_img_url}')
print(f'IMDb Rating: {imdb_rating}')


Title Poster Image URL: https://www.justwatch.com/images/backdrop/302258335/s1440/violent-night/violent-night
IMDb Rating: None


In [ ]:
# Return a dictionary with all the movie data
movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': genres,
    'runtime': runtimes,
    'rating': ratings_list,
    'age_rating': Age_Rating,
    'director_name':directors
}

# Convert to a DataFrame        
Amazonprime_df = pd.DataFrame(movie_data)

In [ ]:
Amazonprime_df

In [ ]:
Amazonprime_df.to_csv('Amazonprime.csv', index = False)

## Getting all Disney+ Movies from 2013-2022

In [ ]:
def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()

    # Send a request to the website
    url = 'https://www.justwatch.com/us/provider/disney-plus/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

# Create empty lists for movie data
urls = get_movies_data()
Movies = []
Released_year = []
ratings_list = []
genres = []
Age_Rating = []
runtimes = []
directors = []

for url in urls:
    # Get the movie title
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    Movies.append(title)
    print(f"Title: {title}")

    # Get the released year
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    Released_year.append(year)
    print(f"Released year: {year}")

    # Get the IMDb rating
    def get_imdb_rating(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
            rating = imdb_tag.text.strip().split()[0]
        except AttributeError:
            rating = 'N/A'
        return rating
    rating = get_imdb_rating(url)
    ratings_list.append(rating)
    print(f"IMDb rating: {rating}")

    # Get the TV age rating
    age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
    if age_rating_div:
        age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    else:
        age_rating = 'N/A'
    Age_Rating.append(age_rating)
    print(f"Age rating: {age_rating}")
    
    #get the movie genres
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
    try:
        genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
    except AttributeError:
        genre = 'N/A'
    genres.append(genre)
    print(f'Genres: {genre}')

    # Get the movie runtime
    runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
    runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    runtimes.append(runtime)
    print(f"Runtime: {runtime}")

    # Get the movie director
    director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
    if director_div:
        director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip()
        directors.append(director_name)
    else:
        directors.append('N/A')
    print(f"Director: {director_name}\n")



In [ ]:
# Return a dictionary with all the movie data
movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': genres,
    'runtime': runtimes,
    'rating': ratings_list,
    'age_rating': Age_Rating,
    'director_name':directors
}

# Convert to a DataFrame        
disney_df = pd.DataFrame(movie_data)

In [ ]:
disney_df

In [ ]:
disney_df.to_csv('disney.csv', index = False)

## Getting all Hulu movies from 2013-2022

In [ ]:
def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()
    
    # Send a request to the website
    url =  'https://www.justwatch.com/us/provider/hulu/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

# Create empty lists for movie data
urls = get_movies_data()
Movies = []
Released_year = []
ratings_list = []
genres = []
Age_Rating = []
runtimes = []
directors = []

for url in urls:
    # Get the movie title
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    Movies.append(title)
    print(f"Title: {title}")

    # Get the released year
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    Released_year.append(year)
    print(f"Released year: {year}")

    # Get the IMDb rating
    def get_imdb_rating(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
            rating = imdb_tag.text.strip().split()[0]
        except AttributeError:
            rating = 'N/A'
        return rating
    rating = get_imdb_rating(url)
    ratings_list.append(rating)
    print(f"IMDb rating: {rating}")

    # Get the TV age rating
    age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
    if age_rating_div:
        age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    else:
        age_rating = 'N/A'
    Age_Rating.append(age_rating)
    print(f"Age rating: {age_rating}")
    
    #get the movie genres
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
    try:
        genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
    except AttributeError:
        genre = 'N/A'
    genres.append(genre)
    print(f'Genres: {genre}')

    # Get the movie runtime
    runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
    if runtime_div:
        runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
    else:
        runtime = 'N/A'    
    runtimes.append(runtime)
    print(f"Runtime: {runtime}")


    # Get the movie director
    director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
    if director_div:
        director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip()
        directors.append(director_name)
    else:
        directors.append('N/A')
    print(f"Director: {director_name}\n")



In [ ]:
# Return a dictionary with all the movie data
movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': genres,
    'runtime': runtimes,
    'rating': ratings_list,
    'age_rating': Age_Rating,
    'director_name':directors
}

# Convert to a DataFrame        
Hulu_df = pd.DataFrame(movie_data)

In [ ]:
Hulu_df

In [ ]:
Hulu_df.to_csv('Hulu.csv',index = False)

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.justwatch.com/us/movie/midsommar'

# Send a request to the website
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the img tag with alt='Amazon Prime Video'
amazon_prime_img = soup.find('img', {'alt': 'Amazon Prime Video'})

# Extract the streaming platform from the alt attribute
if amazon_prime_img:
    streaming_platform = amazon_prime_img['alt']
    print(f"Streaming Platform: {streaming_platform}")
else:
    print("Streaming Platform not found.")


In [ ]:
from bs4 import BeautifulSoup
import requests

def scroll_down(driver):
    # Add your scroll down logic here
    pass

def get_imdb_rating(url):
    # Function to get IMDb rating from a given movie URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
        rating = imdb_tag.text.strip().split()[0]
    except AttributeError:
        rating = 'N/A'
    return rating

def scrape_movie_data(url):
    # Function to scrape movie data from a given URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
    age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip() if age_rating_div else 'N/A'
    genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
    genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip() if genre_div else 'N/A'
    runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
    runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
    director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip() if director_div else 'N/A'
    imdb_rating = get_imdb_rating(url)

    # Extract the streaming platform
    streaming_platform_img = soup.find('img', {'alt': 'Amazon Prime Video'})  # Modify 'Amazon Prime Video' based on the platform
    streaming_platform = streaming_platform_img['alt'] if streaming_platform_img else 'N/A'

    return {
        "Title": title,
        "Released Year": year,
        "Age Rating": age_rating,
        "Genres": genre,
        "Runtime": runtime,
        "Director": director_name,
        "IMDb Rating": imdb_rating,
        "Streaming Platform": streaming_platform
    }

# Example usage:
url = 'https://www.justwatch.com/us/movie/midsommar'
movie_data = scrape_movie_data(url)
print(movie_data)


In [31]:
import pandas as pd
from imdb import IMDb

# Function to get age ratings for all countries for a movie title
def get_all_age_ratings(movie_title):
    ia = IMDb()

    # Search for the movie by title
    search_results = ia.search_movie(movie_title)

    if search_results:
        # Get the first result (most relevant)
        movie = search_results[0]

        # Load detailed information about the movie
        ia.update(movie, info=['main', 'certificates'])

        # Get certificates for all countries
        certificates = movie.get('certificates')

        # Check if certificates is a string, convert it to a list
        if isinstance(certificates, str):
            certificates = [certificates]

        # Create a dictionary to store age ratings for all countries
        age_ratings = {}

        for certificate in certificates:
            # Extract country and rating information
            country, *rating_parts = certificate.split(':')
            rating = ':'.join(rating_parts).strip() if rating_parts else 'Not available'
            age_ratings[country.strip()] = rating.strip()

        return age_ratings
    
    return {'Not available': 'Not available'}

# Your list of movie titles
movie_titles = ['Violent Night', 'Last Flag Flying', 'Everything Everywhere All at Once', 'Last Christmas', 'Honeymoon', 'Better Watch Out', '2036 Origin Unknown', 'The Man Who Invented Christmas', 'A Christmas Carol', 'Extracurricular', 'The Green Knight', 'Butt Boy', 'Vanilla', 'Dead Zone', 'Summer of 85']
# List to store age ratings for all countries
all_age_ratings = []

# Iterate through the movie titles to get age ratings for all countries
for title in movie_titles:
    age_rating_dict = get_all_age_ratings(title)
    all_age_ratings.append(age_rating_dict)
    
    # Print the age ratings for the current movie
    print(f"{title}:\n{age_rating_dict}")

# Create a DataFrame with age ratings for all countries
df = pd.DataFrame(all_age_ratings)

# Add a column for Movie Titles
df['Movie Title'] = movie_titles

# Reorder columns to have 'Movie Title' as the first column
df = df[['Movie Title'] + [col for col in df.columns if col != 'Movie Title']]

# Print the DataFrame
print("\nDataFrame with Age Ratings for All Countries:")
print(df)


2023-12-26 20:35:47,069 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"


Violent Night:
{'Argentina': '16::(with warning)', 'Australia': 'MA15+', 'Belgium': '16', 'Brazil': '16', 'Canada': '13+::(Quebec)', 'Chile': '14', 'Colombia': '15', 'Denmark': '15', 'Ecuador': '15', 'Egypt': '16+', 'Finland': 'K-16', 'France': 'Tous publics avec avertissement', 'Germany': '16', 'Hong Kong': 'IIB', 'India': 'A', 'Indonesia': '17+', 'Ireland': '16', 'Israel': '14', 'Italy': 'VM14', 'Japan': 'R15+', 'Kuwait': '18+', 'Luxembourg': '16', 'Mexico': 'C', 'Netherlands': '16', 'Norway': '15', 'Philippines': 'R-16::(self-applied)', 'Poland': '15', 'Portugal': 'M/16', 'Singapore': 'M18', 'South Africa': '16', 'South Korea': '15', 'Spain': '16', 'Sweden': '15', 'Switzerland': '16', 'Taiwan': '6+', 'Thailand': '15+', 'Turkey': '16+', 'United Kingdom': '15', 'United States': 'R', 'United Arab Emirates': '15+', 'Vietnam': 'C18'}


2023-12-26 20:35:52,565 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"


Last Flag Flying:
{'Argentina': '13', 'Australia': 'M', 'Brazil': '12', 'Canada': '14A::(British Columbia)', 'Colombia': '7', 'Germany': '18::(self-applied)', 'Hong Kong': 'IIB', 'Ireland': '15A', 'Israel': '12', 'Japan': 'G', 'Mexico': 'B', 'New Zealand': 'M', 'Portugal': 'M/14', 'Singapore': 'NC16', 'Spain': '12::(ICAA)', 'Turkey': '13+', 'United Kingdom': '15', 'United States': 'R', 'United Arab Emirates': '18+'}


2023-12-26 20:35:58,724 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"


...

Vanilla:
{'Argentina': '16', 'Australia': 'MA15+', 'Brazil': '14', 'Canada': '14A', 'Ecuador': 'Sin Calificación::(self-applied)', 'Egypt': 'Not Rated::(self-applied)', 'Finland': 'K-12', 'France': 'Tous publics', 'Germany': '16', 'Hong Kong': 'IIB', 'Hungary': '16::(DVD)', 'Iceland': '12', 'India': 'UA', 'Indonesia': '18+::(self-applied)', 'Ireland': '15', 'Israel': '17+::(self-applied)', 'Italy': 'T', 'Japan': 'G::(self-applied)', 'Mexico': 'B', 'Netherlands': '12', 'New Zealand': 'M', 'Nigeria': 'PG', 'Peru': '14', 'Philippines': 'R-18', 'Poland': '12::(self-applied)', 'Portugal': 'M/16', 'Russia': '16+', 'Singapore': 'M18', 'South Africa': '16', 'South Korea': '18', 'Spain': '13', 'Sweden': '11', 'Switzerland': '14::(canton of Vaud)', 'Taiwan': '12+', 'Thailand': 'u 18+::(self-applied)', 'Turkey': '15+', 'United Kingdom': '15', 'United States': 'R', 'United Arab Emirates': '18+::(self-applied)'}


2023-12-26 20:37:01,509 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"


Dead Zone:
{'Argentina': '13', 'Australia': 'M', 'Belgium': '14::(recommended rating)', 'Brazil': '14::(DVD)', 'Bulgaria': 'D', 'Canada': '14+::(tv rating)', 'Ecuador': 'Not Rated', 'Finland': 'K-16', 'France': 'Tous publics', 'Germany': '16', 'Iceland': '16', 'India': 'UA', 'Ireland': '18::(DVD)', 'Italy': 'T', 'Japan': 'PG12', 'Maldives': '18+', 'Mexico': 'C', 'Netherlands': '16::(DVD rating)', 'New Zealand': 'PG', 'Norway': '16', 'Portugal': 'M/16', 'Singapore': 'NC-16', 'South Korea': '18', 'Spain': '13', 'Sweden': '15', 'United Kingdom': '18', 'United States': 'R', 'United Arab Emirates': 'Not Rated', 'West Germany': '16::(theatrical and VHS rating)'}


2023-12-26 20:37:08,564 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"


Summer of 85:
{'Argentina': '16', 'Australia': 'MA15+', 'Belgium': 'KT', 'Brazil': '14::(video rating)', 'Canada': 'Not Rated', 'Chile': '16+', 'Colombia': '15', 'Denmark': '11', 'Ecuador': '16', 'France': 'Tous publics', 'Germany': '12', 'Hungary': '16', 'Iceland': 'Unrated', 'India': 'Not Rated', 'Italy': 'T', 'Japan': 'PG12', 'Netherlands': '12', 'New Zealand': 'R15', 'Norway': '12::(cinema rating)', 'Peru': '14', 'Poland': '12', 'Portugal': 'M/14', 'Russia': '18+', 'Saudi Arabia': 'Not Rated', 'Singapore': 'R21', 'South Korea': '15', 'Spain': '12', 'Sweden': '11', 'Switzerland': '14', 'Turkey': '15+::(self-applied)', 'United Kingdom': '15', 'United States': 'Not Rated'}

DataFrame with Age Ratings for All Countries:
                          Movie Title            Argentina        Australia  \
0                       Violent Night   16::(with warning)            MA15+   
1                    Last Flag Flying                   13                M   
2   Everything Everywhere All at 

In [37]:
import pandas as pd
from imdb import IMDb
import time

# Function to get genres for a movie title
def get_movie_genres(movie_title):
    ia = IMDb()

    # Set a longer timeout (in seconds)
    ia.timeout = 30

    # Search for the movie by title
    search_results = ia.search_movie(movie_title)

    if search_results:
        # Get the first result (most relevant)
        movie = search_results[0]

        # Load detailed information about the movie, including genres
        ia.update(movie, info=['main', 'genres'])

        # Get genres
        genres = movie.get('genres')

        return genres if genres else ['Not available']
    
    return ['Not available']

# Your list of movie titles
movie_titles = ['Hours', 'The Grinch', 'In a World...', 'All the Money in the World', 'Relic', 'The Night Before', 'Bad Timing', 'A Vigilante', 'Last Flag Flying', 'The Infiltrator', 'Cry Macho', 'Genius', 'A Beautiful Day in the Neighborhood', 'This Is the End', 'Silent Night', 'Click & Collect', 'The Little Mermaid', "Don't Breathe 2", "Daddy's Home 2", 'We Still Kill the Old Way', 'War Dogs', 'The Fall of Usher', 'Your Christmas or Mine?', 'Rough Night', 'Nurse 3-D', 'Once Upon a Time… in Hollywood', 'Rubikon', 'The Craft: Legacy', 'When Darkness Falls', 'I Can Only Imagine', 'Dawn Breaks Behind the Eyes', 'A Simple Favor', 'Kin', 'Kinsenas, Katapusan', 'I Spit on Your Grave 2', 'Edge of Tomorrow', 'Rabbit', "Sister Mary's Angel", 'The Tiger', "Daddy's Home", "God's Petting You", 'The Hunger Games: Mockingjay - Part 1', 'Megan Leavey', 'The To Do List', 'A Bad Moms Christmas', 'Pamasahe', 'Love the Coopers', 'Spiral', 'The Long Night', 'Deadly Weekend', 'Ghostbusters', 'How to Be Single', 'Horns', 'The Man Who Saved the World', 'Passengers', 'Sex Tape', 'Mr. Jones', 'My Salinger Year', 'Centigrade', 'Into the Storm', 'The Voices', 'Tahan', 'Val (Feature)', "Franklin's Comedy Book", 'Vanquish', '300: Rise of an Empire', 'Misogynist', 'Honeydew', 'Olympus Has Fallen', 'Hungry Hearts', 'Joe', 'Lorelei', 'Hunt for the Wilderpeople', 'Project Wolf Hunting', 'Carmen & Lola', 'Lucky Bastard', 'Episodes from Apocalypse', 'The Quiet Ones', 'Bae Wolf', 'In the Heart of the Sea', 'Life', 'Melody on Earth', 'The Champions', 'Project Gemini', 'It’s Quieter in the Twilight', '24 Hours to Live', 'American Assassin', 'After Earth', 'Ode to Joy', 'Coma', '11th Hour Cleaning', 'Body Brokers', "Rules Don't Apply", 'The Bromley Boys', 'Triangle of Sadness', 'God of Thunder', 'Beyond Beyond', 'Like Dogs', 'Killing Ground', 'Lapsis', 'Jumanji: Welcome to the Jungle', 'Come As You Are', 'In Search of the Last Action Heroes', 'Kidnapped to the Island', 'The Wedding Guest', 'Operation Avalanche', 'Black Friday', 'Tom Petty, Somewhere You Feel Free', 'The House', 'A Family Man', 'The Parachute Murder Plot', 'I Spit on Your Grave III: Vengeance Is Mine', 'Hail to the Deadites', 'The Big Short', 'Focus', 'Fortress', 'Submission', 'Bohemian Rhapsody', 'Green Room', 'Pinocchio', 'Get a Job', 'A Perfect 14', 'The Otherworld', 'Freeze', 'The Lodger', 'Creators: The Past', 'The Climb', 'Strix', 'Father Figures', 'The Big Hit', 'Cabin Fever', 'All Is True', 'Immanence', 'A Brilliant Young Mind', "Charlie's Angels", 'Sausage Party', 'Home Again', 'The Whale', "The Internet's Own Boy: The Story of Aaron Swartz", 'Gravity', 'Girl/Girl Scene: The Movie', 'Elysium', 'Jiu Jitsu', 'Babysitter Must Die', 'The Little Death', 'Josie & Jack', 'Three Thousand Years of Longing', "Matthew Bourne's Cinderella", 'Clinton Road', 'Here Before', 'My Body Is Not Obscene', 'Come to Daddy', 'Battle for Pandora', 'Amityville Christmas Vacation', 'The Revenant', 'John Wick: Chapter 3 - Parabellum', 'Blood Father', 'Pennywise: The Story of IT', 'Hero Mode', 'Breach', 'The Toll', '377', 'Expired', 'Emperor', 'Collateral Beauty', "Film Stars Don't Die in Liverpool", 'Tuftland', "This Game's Called Murder", 'Bloody Hell', 'The Last of the Winthrops', 'The Pond', 'Held', 'The Hunger Games: Mockingjay - Part 2', 'Night of Something Strange', 'The Other Side of the Ring', 'Run All Night', 'Just Girls', '21 Bridges', 'Terror Trips', '303 Squadron', 'Palm Springs', 'The Killing Tree', 'Flunk: The Sleepover', 'Strip Club Massacre', 'Bill', 'Unfinished Business', 'A Quiet Place', 'Welcome Home', 'Angel Azul', 'Ghost in the Shell', 'The Sixth Secret', 'Rock Camp: The Movie', 'Beyond the Gates of Hell', 'Blue Ruin', 'The Hobbit: The Desolation of Smaug', 'The 5th Wave', 'Rooney', 'Angel Has Fallen', 'Hot Air', 'Fruitvale Station', "Journey's End", '1000km of Chaos', 'Dug Up', 'Women Talking', 'The Nice Guys', 'Ghost Note', "The Devil's Candy", 'Slumber Party Massacre', "Other People's Children", 'Detective Knight: Redemption', 'Pawn Shop Chronicles', 'Lovelace', 'Last Three Days', 'Flames of Fury', 'Sniper: The White Raven', 'Hippopotamus', 'Deinfluencer', 'Horrible Bosses 2', 'Spinster', 'Stereoscope', 'Warsaw: A City Divided', "The Artist's Wife", 'Suspiria', 'The Justice of Bunny King', 'Monster', 'On the Count of Three', 'VHS Massacre Too', 'The Magic Pill', 'The Lady in the Van', 'Sex Doll', 'Sing Street', 'When the Mist Clears', 'Rifftrax: Stitches', 'Hunting Elephants', "I'll Follow You Down", 'Living', 'The Babadook', 'Censor', 'Low Tide', 'Hell or High Seas', 'The Green Knight', 'Summer Camp', 'Tone-Deaf', 'Iliza Shlesinger: War Paint', "The Nanny's Night", 'Nocturnal', 'Plan 9', 'No One Lives', 'Once Upon a River', 'VHS Massacre: Cult Films and the Decline of Physical Media', 'Wildflower', 'The Tomorrow War', 'Once I Was a Beehive', 'Draft Day', 'The Undamaged', 'The Last Scout', 'Hotel Transylvania: Transformania', 'Congratulations!', 'Monolith', 'Beauty Factory', 'The Last Stop', 'The Matrix Resurrections', 'Secret Societies', 'The Last Heist', 'Susie Q', 'The White Orchid', 'Look Away, Look Away', 'Armstrong', 'Night Kaleidoscope', 'Greenland', 'The Hating Game', 'Before Midnight', 'The Haunting of Pendle Hill', 'Strangerland', 'The Long Shadow', 'Above Ground', 'When Claude Got Shot', 'Going in Style', 'Testament of Youth', '安娜', 'Girl at the Window', "The House on Devil's Road", 'You, Me and Him', 'The Christmas Candle', 'In the Forest', 'Follow the Dead', 'Altered Hours', 'A Perfect Day', 'Her Smell', 'Native', 'Beautiful Beings', 'Still Human', '2020: The Dumpster Fire', 'Happy Christmas', 'The Wall', 'The Formula', 'A Beauty & the Beast Christmas', 'I Am Big Bird: The Caroll Spinney Story', 'Standing Up, Falling Down', 'A Few Cubic Meters of Love', 'Fresh Dressed', 'The World We Knew', 'Loving Ibiza', 'Walkaway Joe', 'The Possessed', 'Heavy Trip', 'The Narcissists', 'The Phoenix', 'The Man Who Sold His Skin', 'A Patch of Fog', 'Good Luck to You, Leo Grande', 'Station to Station', 'The Wailing', 'Band of Robbers', 'The Red Book Ritual', 'Parable', 'Glassland', 'Wild Honey Pie!', 'Alien Invasion: S.U.M.1', 'Rams', 'RoboCop', 'The Vault', 'Fan', 'Amsterdam', 'Butt Boy', 'Tomorrow Ever After', 'The Longest Week', 'Salinger', 'The Last Laugh', 'The Fanatic', 'The Pebble and the Boy', 'I Am Woman', 'Feast of the Seven Fishes', 'Lost Girls & Love Hotels', 'Solid Rock Trust', 'Exegesis Lovecraft', "I Think We're Alone Now", 'Gangster Squad', 'Vice', 'Broadcast', 'Let Us In', 'The Resort', 'The Belko Experiment', 'Away From Here', 'We Broke Up', 'Results', 'The Wolf Hour', 'The Stairs', 'The Stranger', 'Unacknowledged', 'The Tape', 'Deadly Seduction', "Darlin'", 'Mutant Blast', 'Broadcast Signal Intrusion', 'Kill Your Darlings', 'Fifty Shades of Fetish Model', 'The Owners', 'Sword of God', "Pretending I'm a Superman: The Tony Hawk Video Game Story", 'Lady Lawman', 'El número nueve', 'The Kindness of Strangers', 'The Contractor', 'Here Are the Young Men', 'Dreaded Light', 'Entanglement', 'Pledge', 'Island Zero', 'Death Pool', 'CRSHD', 'Outback', 'Dashcam', 'The Hungover Games', 'Zombieland: Double Tap', 'Free Ride', 'Hell House LLC III: Lake of Fire', 'Birdman or (The Unexpected Virtue of Ignorance)', 'Easter Bunny Massacre', 'Voyeur', 'Hellblazers', "Don't Call Me Bigfoot", 'Landmine Goes Click', 'Love Hurts', 'Deepwater Horizon', 'All Is Bright', 'Iravin Nizhal', 'Daddy Issues', 'Short Term 12', 'Square One', 'Alpha', 'Grand Isle', 'Close Encounters of the Fifth Kind', 'The Message', 'Patio', 'This Beautiful Fantastic', 'The Grand Bolero', 'The Long Walk', 'Thirteen Lives', "The King's Daughter", 'The Big Sick', 'Islanders', 'Wither', 'Akron', "Don't Click", 'Pescador', 'Dead in the Water', 'Safe Haven', 'Softness of Bodies', 'Roll Red Roll', 'The Pine Barrens', 'The Last Warrior', 'The Cheerleader Sleepover Slaughter', 'Poker Night', 'A Guide to Second Date Sex', 'Bullet Head', 'Bombshell', 'Diani and Devine Meet the Apocalypse', 'Burn Burn Burn', 'Hellboy', 'Doors', 'Ditched', 'Piranhas', 'The Legend of Hawes', 'A Savannah Haunting', 'Mr. Dynamite: The Rise of James Brown', 'Beyond the Wasteland', 'Ivanna', 'First Love', 'The Adventures of Paula Peril', 'Sideworld: Damnation Village', 'The Beast', 'Nightmare Wedding', 'Argentina, 1985', 'Risen', 'Mindhorn', 'Deep Water', 'The Pilot: A Battle for Survival', 'The East', 'Working Man', 'Immortal', 'Curse of Crom: The Legend of Halloween', 'Line of Duty', 'Jallikattu', 'After Ever Happy', 'Bones and All', 'The Carpenter', 'Emelie', 'The Christmas Train', 'Kursk', 'Buffaloed', "Assassin's Creed", 'The Last Rescue', 'Stacked', 'Foster Boy', 'Sideworld: Terrors of the Sea', 'The Rising Hawk', 'Secrets on Sorority Row', 'St. Agatha', 'Demonic', 'Baking All the Way', 'Dead in a Week (Or Your Money Back)', 'Stranger Fruit', "The High Frontier: The Untold Story of Gerard K. O'Neill", 'Shattered', 'The Big Take', 'The Upside', 'Asterix: The Mansions of the Gods', 'KKN di Desa Penari: Luwih Dowo, Luwih Medeni', 'Manbaby', 'Jack of the Red Hearts', 'Hunter Killer', 'The Purge', 'Supremacy', 'Runner Runner', 'Fist Fight', 'Arctic', 'Savage', 'The Misfits', 'The Ones Below', 'Blood Vessel', 'Impossible Dreamers', 'Evil Eye', "Thee Debauchery Ball: Director's Cut", 'The Ledge', 'Changeland', 'Division 19', 'The Boat', 'Walk of Shame', 'Jack the Giant Slayer', "Devil's Workshop", 'Flight/Risk', 'It Came from Below', 'Dave Stevens: Drawn to Perfection', 'I.T.', 'The Hippopotamus', 'Ash Is Purest White', 'Django', 'Kick Out the Jams: The Story of XFM', 'Superior', 'Videoman', 'Neon', 'Possum', "Italy's Mystery Mountains", 'Tang', 'Shark Killer', 'Honour', 'Boyfriend Killer', 'The Son', 'Prince of Muck', 'Billy Graham: An Extraordinary Journey', 'Prevenge', 'Bloodthirsty', 'Last Shift', 'Swelter', 'The Girl Who Killed Her Parents', 'Dead by Dawn', 'The Ash Lad: In the Hall of the Mountain King', 'Bar Fight', 'Horror in the High Desert', 'Finding Steve McQueen', 'Creating Christ', 'Haunting of the Mary Celeste', 'The Unkind', 'The Intern', 'Shotgun Wedding', 'Soorarai Pottru', 'Rot', 'Come Away', 'H Is for Happiness', 'Chasing Happiness', 'The Haunted Hotel', 'Pixels', 'Vollendet', "The Party's Just Beginning", 'Jurassic Domination', 'Flirts', 'Sinister Switch', 'The Modern Way', 'Christmas in Paradise', 'Floating!', 'Oildale', 'Whispers', 'Scarecrow', 'Red Carpet', 'Pelé: Birth of a Legend', 'Demons at Dawn', 'Age of Summer', 'Gates of Darkness', 'A Boy Called Sailboat', 'Goatling: Son of Satan', 'Depraved', 'Dark Beacon', 'Europa', 'The Curse of Hobbes House', 'Spotlight on Christmas', 'Softie', 'Shelter', 'Catherine Called Birdy', 'Once Upon a Time in... Hollywierd', 'Intersect', 'A Hero', 'Ponniyin Selvan: Part I', 'Borrego', 'Quintessentially British', 'Strangeways Here We Come', 'The Preppie Connection', 'Winterskin', 'Paul Blart: Mall Cop 2', 'Electric Jesus', 'The Journey', 'Proximity', "Fortress: Sniper's Eye", 'Green White Green', 'The Protégé', 'The Postcard Killings', 'Bellbottom', 'A Dark Place', 'Killbird', 'What We Did on Our Holiday', 'DriverX', 'Everything in the End', 'I Care a Lot', 'The Illegal', "All the Devil's Men", 'JugJugg Jeeyo', 'Gatlopp', 'What Have We Done to Deserve This?', 'Above the Shadows', 'Creed II', 'Le Brio', 'Black Hollow Cage', 'Here Comes Hell', 'Brutus vs Cesar', 'Brothers', 'The Frontier', 'Songbird', 'The Badge, the Bible, and Bigfoot', 'Dear Zoe', 'Lapwing', 'Parallel Lines', 'Skyfire', 'Babes with Blades', 'Revenge Best Served Chilled', 'Return to Me', 'Killing Your Daughter', 'Hostage Radio', 'Rise', 'Overdose', 'Scottish Mussel', 'Aquarium of the Dead', 'A Lethal Lesson', 'The Man Who Killed Don Quixote', 'I Am Norman', '3 Days to Kill', 'The Only Living Boy in New York', 'Dating & New York', 'God Help the Girl', 'Top End Wedding', 'Trust', 'After the Dark', 'Day X', 'Kill Ben Lyk', 'Repeat', 'Borat Subsequent Moviefilm', 'Babe Beach', 'Peridot', 'Bad Day for the Cut', 'Shamshera', '2.0', 'An Ordinary People', 'LX 2048', 'Siren', 'Metamorphoses', 'Jellyfish', 'Benjamin', 'Boss Level', 'VFW', 'Take Down', 'HIT', 'The Monster', 'Get Back', 'Douche', 'Get Hard', 'Escape from Death Block 13', 'Desecration', 'Guns Akimbo', 'Corona Depression', 'Kill Chain', 'One Last Thing', 'Headless Horseman', 'From the Depths', 'Cold Brook', 'Student Body', 'Wonderstruck', 'Transhuman', 'The Scrapper', 'Geostorm', 'The Sea', 'The Adventures of Thomasina Sawyer', 'Heropanti 2', 'The Last Five Years', 'Sobibor', 'Birds of Paradise', 'The Unknown Runner', 'The Choice', 'The Drowning of Arthur Braxton', '3 Way Junction', 'Battle on Buka Street', 'Viruman', 'Bundy and the Green River Killer', 'The Take', 'Vendetta', 'La semilla del silencio', 'Molly', 'Santa Fake', 'Above Ground Level: Dubfire', 'Body at Brighton Rock', 'Postal', 'The Wind', 'Raatchasi', 'The Bright Side', 'Havenhurst', 'The Haunting of Margam Castle', 'The Last Man on the Moon', 'Love on the Slopes', 'Marie Curie, The Courage of Knowledge', 'Jolt', 'Vendhu Thanindhathu Kaadu', 'First Date', 'Polterheist', '365', 'Ananya', 'Haunted', 'The Runners', 'Slipaway', 'Aadai', '4:4:44', 'Death in Texas', 'Approaching Shadows', 'Good Night Oppy', 'Jack Lantern', 'Whiplash', 'Headgame', 'Loro 1', 'Cold Case', 'Hyena Road', 'Made Guys', "There's Something About Mario", 'Inheritance', 'Memory', 'Iron Mask', 'Beyond the Woods', 'Laddie: The Man Behind the Movies', 'COVID-21: Lethal Virus', 'The Map of Tiny Perfect Things', 'Bayou Caviar', 'The Greasy Hands Preachers', 'Zebra Girl', 'Peterloo', "Potter's Ground", 'Batla House', 'A Cure for Wellness', 'The Appearance', 'The Sonata', 'The Diabolical', 'Escape from Mogadishu', 'Putham Pudhu Kaalai', 'Looks That Kill', 'The Rest of Us', 'Last Rampage', 'Killing Gunther', 'All Nighter', 'Run', 'Boulevard', 'Velvet', 'Unda', 'Zoolander 2', 'Big Legend', "Never Goin' Back", 'Come to Harm', 'Follow Me', 'Dead Man Down', 'DannyBoy', 'The Burying Party', 'The Damned', '48 Christmas Wishes', 'Sarpatta Parambarai', 'Cardboard Gangsters', 'Hunt vs Lauda: The Next Generation', 'First Born', 'The Convent', 'Project Ithaca', 'Farmer of the Year', 'Portals', 'Ripper Untold', 'Triple Threat', 'Robert Reborn', 'Shane', 'The Dark Tower', 'Aloha', 'Penguin', 'Hustle', 'Knocking', 'Grave Intentions', 'The Man in the Hat', 'The Furnace', 'One Night Off', 'Rogue Warfare', 'Radhe Shyam', 'The Brave', 'Evanescence: Embracing the Bitter Truth', "Fisherman's Friends: One and All", 'Crossfire', 'Gatham', 'Hytte', 'Bumperkleef', "Zoey's Extraordinary Christmas", 'Clara', 'Lowlife', 'On the Line', "Maussan's UFO Files", 'Swim', 'Snowkissed', "The Eagle's Nest", 'Boy in the Corner', 'Pass Over', 'Digging to Death', 'Mad to Be Normal', 'Rage', 'Bloodline', 'The Livingston Gardener', 'Escape Room', 'Beyond White Space', 'Light in the Dark', 'The Saturn V Story', 'The Broken Hearts Gallery', 'School of the Damned', 'Antrum', 'Valley Girl', "While We're Young", 'Ashes in the Snow', 'City of Tiny Lights', 'Boy Meets Girl', 'Master Z: Ip Man Legacy', 'The Worst Thing', 'Parallel', 'Woodland', 'Seven Weeks', 'Deep Hatred', 'About Fate', 'Brexitannia', 'Darbar', 'Women', 'Bad Match', 'Dave Made a Maze', 'Peace Officer', 'Most Beautiful Island', 'The Hatred', 'Victor Crowley', "The Watchmaker's Apprentice", 'The Courier', 'Papillon', 'Kajaki', 'SGT. Will Gardner', 'Bharat', "Tom Clancy's Without Remorse", 'Samrat Prithviraj', 'The Haunting of Sharon Tate', 'The Aeronauts', 'Dragon Knight', 'Never Surrender: A Galaxy Quest Documentary', 'Hidden Turkey', 'D-Railed', 'Mayday', 'Tokyo Rising', 'W/O Ram', 'Nerve', 'Killer Legends', 'Mendarat Darurat', 'Encounter', 'When Jeff Tried to Save the World', 'Myth: Bigfoot Hunters', 'Gold', 'True Story', 'Death Alley', 'Crown Heights', 'Pale Star', 'The Fourth Musketeer', 'RiffTrax: Lycan Colony', 'I Write My Life', 'Blackway', 'Cassette: A Documentary Mixtape', 'The Wee Man', '2149: The Aftermath', 'Sound of Metal', 'Aura', "Kendrick Lamar's The Big Steppers Tour: Live from Paris", 'Witches in the Woods', 'Found.', 'Dragonfly', 'Eternal Winter', 'Raja the Great', 'Rocketry: The Nambi Effect', 'Forte', 'Yashoda', 'Sidelined', 'The Utah Cabin Murders', 'Bachchhan Paandey', 'The Lie', 'Jolsobi', 'Killerman', 'Nerdland', 'Behind You', 'Spy Intervention', 'Shylock', 'The Way You Look Tonight', 'The Atoning', 'Irrefutable Proof', 'Country Hard', 'Phantom', 'Ride', 'Assassinaut', 'Kick Like Tayla', 'Stallone: Frank, That Is', '3022', "Straight A's", "Daylight's End", 'Break Every Chain', 'All Eyes on You', 'Marcus', 'Good Newwz', 'HoneyBee', 'Hoodman', '93 Days', 'K. D.', 'Lore', 'Chronically Metropolitan', 'What Lies Below', 'Vengeance Is Mine', 'Bully', 'Sold Out', 'D.L. Hughley: Clear', 'Dabangg 3', 'The Last Conception', 'The Rental', 'Dementamania', 'Wildlife', 'Kin Dread', 'Chick Fight', 'Inherit the Viper', 'Backfire', 'Captain Sabertooth and the Treasure of Lama Rama', 'Monster High: The Movie', 'Beyond the Law', 'Deadware', 'Rolling Stone: Life and Death of Brian Jones', 'Cobra', 'Payday', 'Back in the Day', "Henry Gamble's Birthday Party", 'Tammy', 'Thorp', 'Unit Eleven', 'Beaus of Holly', 'Witch Hunt', 'Make Us Dream', 'Dachau Liberation', 'The Dissident', 'Six Three Times', 'Man of the Match', 'A Haunting on Dice Road: The Hell House', 'Incarnation', 'The Arbitration', 'Mary Loss of Soul', 'It Takes Three', 'Mayor Pete', 'Wanted - Matloubin', 'Day of Reckoning', 'Rebel in the Rye', 'Silence Patton', 'Romance Retreat', 'Honest Thief', 'Miles Jupp: Fibber in the Heat', 'I Love America', 'White Island', 'Darken', 'Songbirds', 'Rachel Feinstein: Only Whores Wear Purple', 'Coldwater', 'The Witch of Kings Cross', 'QT8: The First Eight', 'Brakes', 'The Sound of 007', 'Dying of the Light', 'Thriller', 'The Last Survivors', 'OtherLife', 'Ishq Brandy', 'Mo', 'Bingo Hell', 'Historias lamentables', 'The Nest', 'Koi Jaane Na', 'Pyewacket', 'Luz: The Flower of Evil', 'Perfect Sisters', 'You Are My Problem', 'Greener Grass', 'Ice Guardians', 'The Weight of Gold', 'Paradise Cove', 'Samaritan', 'Tom Segura: Completely Normal', 'The Black Book', 'Sir Alex Ferguson: Never Give In', 'Tucked', 'Kuruthi', 'Maverick: Manhunt Brazil', 'Ammu', 'Super Duper Alice Cooper', 'Gehraiyaan', 'Thank God', 'The Accidental Spy', 'A Brixton Tale', 'When Love Happens', 'The Santa Box', 'Venky Mama', '3 Hearts', 'Crazy, Rich and Deadly', 'Werewolf Castle', 'The Heiress', 'Eye of the Beholder: The Art of Dungeons & Dragons', 'Baked in Brooklyn', 'Unknown Visitor', 'Meppadiyan', 'Kadaram Kondan', 'Elephant Song', "Don't Go", 'CT Fletcher: My Magnificent Obsession', 'IPC 376', 'Plan A', 'Incoming', 'The Frame', 'The Other Wife', 'Wedding Doll', 'The Last Descent', 'Karnan', 'Savyasachi', 'Generation Wealth', 'Marry Me at Christmas', 'Seymour: An Introduction', 'Transference: Escape the Dark', 'Three Days in Havana', 'Kaleidoscope', 'Palestine', 'Saltwater: The Battle for Ramree Island', 'Detective Byomkesh Bakshy!', '7 Deadly Sins', 'Alien Outbreak', 'The Town that Dreaded Sundown', '13 Minutes', 'Girls Like Magic', 'The Drone', "You've Been Trumped Too", 'Gang Leader', 'Peripheral', 'The Bezonians', 'Amar Akbar Anthony', 'Alien Code', 'The Witch Files', 'Vangelis and the Journey To Ithaka', 'Sanctuary: Population One', 'Monster Island', 'Dybbuk: The Curse Is Real', 'Anuvab Pal: Alive at 40', 'Trace', 'The Ice Forest', 'About Us', 'Cherry Pop', 'Chal Mera Putt', 'Leopardi', "Devil's Revenge", 'The Shed', 'Still the Water', "Don't Worry, He Won't Get Far on Foot", 'WWII: The Long Road Home', 'The Cleansing', 'Incidental Characters', 'The Set Up', 'Infernum', 'Home Sweet Home', 'Drawn Into the Night', 'Shadow in the Cloud', 'Apache Stays Apache', 'Zane Lamprey: Tender Looks', 'Planet Dune', 'Ibiza Undead', 'Anonymous Animals', 'Helix', 'Pretty Little Stalker', 'Brighton', 'Up North', 'Big Kill', 'Lurking Woods', 'The Sand', 'Miami', 'Avatara Purusha: Part 1', 'Ted K', 'Saani Kaayidham', 'Trick', 'Easy Does It', 'Alistair1918', 'Benedict Arnold: Hero Betrayed', 'Shakuntala Devi', 'First Signal', 'Nanny', 'My Pet Dinosaur', 'Making Monsters', 'Coven', 'I Am Not a Serial Killer', 'Knuckle City', 'Alone', 'Concrete Blondes', '#UNFIT: The Psychology of Donald Trump', "All'alba perderò", 'Kingdom of the Dinosaurs', 'The Death of April', "Devil's Island", 'The Commuter', 'The Ice Cream Truck', 'Airaa', 'Love on the Sidelines', 'Cowboys vs. Dinosaurs', 'A Girl Like Grace', 'Honsla Rakh', 'RX 100', 'Stalled', 'Borealis', 'The Professor and the Madman', 'Mistaken', 'Pranaam', 'Meet Cute', "You're Killing Me", 'Gwen', 'All This Mayhem', 'Savaged', 'Noah', 'The Purge: Anarchy', 'Mind Games', 'Naane Varuvean', 'Souvenir', 'Wish You Well', 'Encounter', 'Crone Wood', 'Jekyll and Hyde', 'Open Windows', 'Samba', 'And They Say I Am the Crazy One', 'Head Count', 'Kaappaan', 'The House of Violent Desire', 'Drishyam 2', 'Mustang Saviors', 'Frances Ferguson', 'Arsène Wenger: Invincible', 'Walking Out', 'Evil Takes Root', 'Lullaby', 'Aurora', 'Untamed Romania', 'Triple Seat', 'Time', 'The Curse of Professor Zardonicus', 'A Reckoning', 'Bone Tomahawk', 'Redemption', 'Corrective Measures', 'Royal Jelly', 'The Manor', 'Rammstein: Paris', 'Conjoined', 'Goblin', 'From Us to Me', 'Gold Statue', "Everyone's Going to Die", 'Project Gutenberg', 'Blackwater', 'Teenage Ghost Punk', 'The Brawler', 'The Pagan King', 'Legendary: Tomb of the Dragon', "I'm Dreaming of a White Doomsday", 'Malik', 'Elizabeth: A Portrait in Part(s)', 'The Tent', 'The Architect', 'The Wicked', "America's Forgotten", 'Clownado', 'After the Pandemic', 'Infinity Chamber', 'Bigfoot Wars', 'Sarileru Neekevvaru', 'The Lotus', 'Ouija Hosts', 'The Passion of Augustine', 'Remnants of a Disaster', 'Yaanaa', 'Killers Anonymous', 'The Legend of the Christmas Witch', 'Laal Rang', 'The Boy from Medellín', 'All This Panic', 'CBGB', 'Go North', 'Suburra', 'Honey Boy', 'UPSIDEdown', 'The Eighth Clause', 'Forsaken', 'Sachertorte', 'Bridal Wave', 'Addicted to Sheep', 'The Weeknd x The Dawn FM Experience', 'Cold Ground', 'Two 4 One', 'Mission of Honor', 'Black Label Society: Unblackened', 'Mahaan', 'Dad Rocks', 'Jeruzalem', 'We Need to Do Something', 'Rogue Warfare: Death of a Nation', 'The Last Photograph', 'Hot Seat', 'Winter Brothers', 'Assault on VA-33', 'Pataakha', 'Burning', 'Wiener-Dog', 'Being the Ricardos', 'Phony', 'Collider', 'Bad Comments', 'Shershaah', 'Black Widows', 'Rachel Bradley: Alpha Chick', 'Non-Stop', 'Kids in Love', 'Fatal Beauty', "Jeremy Scott: The People's Designer", 'Saving Jaws', 'Secrets Exposed', 'Black Box', 'Dead Again', 'The Last Witness', 'Another You', 'Locked In', 'Wolves of War', 'We Still Steal the Old Way', 'Good After Bad', 'Jawaan', 'The Penalty', 'C U Soon', 'Chappaquiddick', 'Geetha', 'Fear and Loathing in Aspen', 'The Curse of Buckout Road', 'Animator', 'The Legend of Barney Thomson', 'Rabbit School: Guardians of the Golden Egg', 'King Arthur and the Knights of the Round Table', 'Sandeep Aur Pinky Faraar', 'The Book of Love', 'Ip Man 3', 'A Good Marriage', 'The Last Case of August T. Harrison', 'The Purge: Election Year', 'Killer Piñata', 'Bear-Like', 'Borealis', 'Kalank', 'Invasion', 'The Rise of the Krays', 'A Man Named Scott', 'The Division: Agent Origins', 'Better Days: The Story of UK Rave', 'All In: The Fight for Democracy', 'This Is Not a Movie: Robert Fisk and the Politics of Truth', 'West of Eden', 'Narappa', 'Kickboxer: Vengeance', 'In-Lawfully Yours', 'Dark Was the Night', 'Soundtrack to Sixteen', 'The 24th', 'Underwater Federica Pellegrini', 'Shepherd', 'Iruttu Araiyil Murattu Kuthu', 'Bigil', 'The Tender Bar', 'Love Dot Com: The Social Experiment', '10 Days In Sun City', 'Moonwalkers', 'Steven Wilson: Home Invasion - In Concert at the Royal Albert Hall', 'Beyond the Lights', 'I Want You Back', 'Into the Forest', 'Attack of the Lederhosen Zombies', 'Sherni', "Let's Kill Ward's Wife", 'What the Waters Left Behind', 'We Go in at Dawn', 'Sher Shivraj', 'The Boy Who Killed My Parents', "Don't Hang Up", 'Flesh City', '2025 Armageddon', 'Run Sweetheart Run', 'K.G.F: Chapter 1', 'Holmes & Watson', 'Sugar Mountain', 'Home Run', 'Viking Destiny', 'The Book of Daniel', 'Prassthanam', 'Gnaw', 'Desconectados', 'Givers of Death', 'The Jack in the Box', 'The Final Wish', "You're Not Alone", 'Rising Wolf', 'Juang', 'A Killer Next Door', 'The Grotto', 'Blaze', 'Abominable', 'A Rising Tide', 'Lost in the Living', 'Citizen K', 'Dune', "Ryan James: I'm Fine", 'Pada', '1BR', 'Human Animals', 'Dog Years', 'Dawn', 'The Real Taylor Swift: Wild Dreams', 'The Ice Road', 'Daddy', 'Bandit', 'The Sleep Experiment', 'Marcus', 'Alpha Code', 'The Orange Man', 'Surviving Gaza', 'Ambili', 'Abigail Haunting', 'Waiting for the Barbarians', 'Majili', 'Central Park', 'Broken Darkness', 'The Extraordinary Journey of the Fakir', 'The Island', 'Wait for Your Laugh', 'Lucifer', 'Bound to Vengeance', 'Blackstock Boneyard', 'Aruvi', 'Broil', 'Vintage Tomorrows', 'Youth in Oregon', 'Joji', 'The Last Shift', 'Little Tickles', 'Ashgrove', 'Legacy of Lies', 'Black Box', 'Bryan Callen: Complicated Apes', 'A Snapshot of Forever', 'The Courtship', 'Lavender', 'Skateshop', 'Shark Waters', 'AmeriGeddon', 'Pulling Push Doors', 'Chasing Gold', 'The Killing Field', 'Blurred Lines: Inside the Art World', 'Arjun Patiala', 'Sardaarji', 'Fall City', 'The Vast of Night', 'Jessica', 'Acts of Violence', 'Kannadi', 'Haunted 3: Spirits', 'The Last Letter', 'Mindcage', 'Chasing the Dragon', 'The Lodgers', "Dancin' It's On!", 'The Darkness', 'Goodnight Mommy', 'Higher Power', 'The Trials of Cate McCall', 'Instant Death', 'Inheritance', 'Way of the Cross', 'Pitbull', 'Playhouse', 'Echoes of Fear', 'The Witness', 'Bhoot: Part One - The Haunted Ship', 'Spoken', 'One Cut Two Cut', 'Men who wanted to climb a mountain over 8000 meters', 'Master', 'Super Size Me 2: Holy Chicken!', 'We Are Triathletes', 'Christmas Pen Pals', 'Bully', 'The Family', 'Dear Comrade', 'Eating Our Way to Extinction', 'Hello Charlie', 'Goodachari', 'Gold Dust', 'Soft Lad', 'Film Fest', 'Love, Kennedy', 'Finding Joy', 'Ash & Dust', 'Sita Ramam', 'The Street', 'Black Circle', 'Blood on Her Name', "Anything's Possible", 'How We Got Gay', 'Rammstein in Amerika', 'Operation Overlord', 'Electronic Dance Music - A Top Export from Germany', 'Devotion', 'Edge of Extinction', 'M.O.M. Mothers of Monsters', 'Against the Night', 'Sanctuary', "Apollo Astronauts: Training NASA's Moon Men", "Everybody's Talking About Jamie", 'Kaala', 'Hawa', 'Macho Macho Man', 'Time Freak', 'Golden Dawn Girls', 'The Dating List', 'A.I. Rising', 'Fuku-chan of FukuFuku Flats', 'McCanick', 'Grey Wolf: The Escape of Adolf Hitler', 'Lazarus', 'Scaffolding', 'The Sleeper by the River', 'Virus', 'Saunkan Saunkne', 'Tom Papa: Freaked Out', 'Doblemente Embarazada 2', 'Metal Heart', 'Behind the Curve', 'The Transparent Woman', 'Midnight Screening', 'Untitled Horror Movie', 'Christmas Incorporated', 'Sui Dhaaga - Made in India', 'Tom Papa: Human Mule', "Hannah: Buddhism's Untold Journey", 'Wildcat', 'Fucking Drama', 'Miss Juneteenth', 'Occupation: Rainfall', 'Theatre of Fear', 'Thank You', 'High Tech Airport', "A Shoe Addict's Christmas", 'Moms at War', 'Dauntless: The Battle of Midway', '2eleven', 'Human Zoo', '1 Buck', 'Gleason', 'Jackpot', 'Once Upon a Prince', 'Ghost', 'Meth Head', 'Shark Huntress', 'A Thousand Times Good Night', "My Dad's Christmas Date", 'House Shark', 'Yeh Jawaani Hai Deewani', 'Index Zero', 'Running for Good: The Fiona Oakes Documentary', 'Vikram Vedha', 'Suriname', 'The Surprise', 'Viking Blood', 'Jayeshbhai Jordaar', 'Soul to Keep', 'Blame', 'Nemesis', 'The Wedding Trip', 'Seven Devils', 'The Green Sea', 'Viking Quest', 'Haunted 2: Apparitions', 'Muddy', 'Marakkar: Lion of the Arabian Sea', 'Equilibrium', 'Rustum', 'Curse of the Nun', 'Maja Ma', 'The Protector', 'Ayyappanum Koshiyum', 'I Am Vengeance', 'Rose Plays Julie', 'Escape from Pretoria', 'Wasteland', 'Mitron', 'Doe', 'Jack N Jill', 'Chicken People', 'Vengeance', 'Saaho', 'Shivaay', 'Elon Musk: The Real Life Iron Man', 'Someone Marry Barry', 'Transfert', 'Sarkaru Vaari Paata', 'The Green Park', 'Incision', 'Dalida', 'Zoo', 'The Secret Life of the Cruise', 'Between the Devil and the Deep', "Mary J. Blige's My Life", 'Detective Knight: Rogue', 'F2: Fun and Frustration', 'Making Babies', 'Pink Wall', 'LAW', 'The Curse of Halloween Jack', 'The Blind Spot', 'Ash and Bone', 'Pablo', 'Evil Never Dies', 'Nails', 'A Death in the Gunj', 'The Class of ‘92', 'American Fighter', 'Pitching Tents', 'Cupid', 'Sultan', 'Twilight of the Yakuza', 'Boston2Philly', 'We Love Paleo', 'Banana Island Ghost', '33 & Beyond: The Royal Art of Freemasonry', 'Pink: All I Know So Far', 'Newlyweeds', 'Big Boys Don’t Cry', 'Max', 'Red Obsession', 'Hippi', 'Whisper', 'Iddari Lokam Okate', 'Down Dog', 'Keep On Keepin’ On', 'The Internship', 'Stevie D', 'Notebook', 'After We Fell', 'Hope Lost', 'Vellai Pookal', 'Sita', 'Power of Grayskull: The Definitive History of He-Man and the Masters of the Universe', 'Road', "The Pilgrim's Progress", 'HIT: The 2nd Case', 'Entourage', 'Ping Pong Summer', 'Somm: Into the Bottle', 'Dear Tenant', 'One Night in Miami...', 'Big Sur', 'Rise of the Footsoldier: Origins', "The Demon's Waltz", 'Jexi', 'Boys in the Trees', 'The Load', 'Transition', 'Let Me Fall', 'Another Forever', 'Dark Seduction', 'Ideal Home', 'Muirhouse', 'Halloweed', 'Christmas with a Crown', 'Island Songs', 'Untimely', 'Boar', 'Manikarnika: The Queen of Jhansi', 'Gulabo Sitabo', 'Listen', 'Concrete Plans', 'Blood Bags', 'Letters from Baghdad', 'Time Is Up', 'Lizzie', 'The Hungry', 'Death Factory', "A Stork's Journey", 'Barista', 'Golmaal Again', 'The Man Who Killed Hitler and Then the Bigfoot', 'A Sunday Horse', 'UFO Conspiracies: The Hidden Truth', 'Beast of Burden', 'The Forbidden Note', 'Fatal Instinct', 'Marvellous', 'Pathonpatham Noottandu', 'Then Came You', 'Shangri-La: Near Extinction', 'Prisoner of Love', 'Seven Stages to Achieve Eternal Bliss', 'Crawlspace', 'Waiting for You', 'The Clockwork Girl', 'We Still Say Grace', 'Barun Rai and the House on the Cliff', 'Yes, God, Yes', 'K.G.F: Chapter 2', 'Made in Italy', 'A Guide to Dating at the End of the World', 'Horizon Line', 'The Magical Christmas Tree', 'Bring Him Back Dead', 'Who Owns Nature?', 'SPiN', 'The Girl Who Believes in Miracles', 'The Last 24 Hours: Elvis Presley', 'Galwakdi', 'Heir', 'Come True', 'Vinaya Vidheya Rama', 'Beyond Bollywood', 'To Let', 'Terror on the Prairie', 'The Reunited States', 'Sharkwater Extinction', 'Little Lies', 'Thappad', 'Show Me What You Got', 'You Must Be Joking', 'Nizhal', 'A Great North Christmas', 'Kim Dotcom: Caught in the Web', 'Cold Skin', 'Lasso', 'Officer', 'Hot Guys with Guns', 'The Beauty of Love', 'Leslie Caron: The Reluctant Star', "Brother's Day", 'Katherine of Alexandria', 'Zakir Khan: Kaksha Gyarvi', 'Jawbone', 'By Light of Desert Night', 'The President', 'Nosferatu.com', 'Emergency', 'The Snail and the Whale', 'Bounty Killer', 'Flashback', 'It Felt Like Love', 'October', 'Ghajinikanth', 'Frequencies', 'Pailwaan', 'The Haunted', '14 Cameras', 'Fire Twister', 'The Tiger Who Came to Tea', 'Alvin and the Chipmunks: The Road Chip', 'Monica and Friends: Lessons', 'Cinderella', 'Homestay', 'Tiffany Haddish: She Ready! From the Hood to Hollywood!', 'Mamangam', 'Wildfire', 'Fossil', 'Weaponized', 'Woe', 'Kabullywood', 'Monster Portal', 'The Connection', 'Prey', 'Sye Raa Narasimha Reddy', 'Avane Srimannarayana', 'A Stranger in Paradise', 'Fair Play', 'Get Duked!', 'Hotel Limbo', 'Bleeding Steel', "I'm Your Woman", 'Promise at Dawn', "H.P. Lovecraft's Witch House", 'Alta Vista', 'King Cnut', 'Thiruchitrambalam', 'Bhaskar Oru Rascal', 'Deadly Crush', 'Christmas in the Highlands', 'East Side Sushi', 'Malayankunju', 'Murder RX', 'Delicious', 'The Ways of Man', 'Giddy Stratospheres', 'Late Night Double Feature', 'Tatu', 'Gali Guleiyan', 'Student of the Year 2', "Yalla Aa'belkon: Single, Married, Divorced", 'LIFE·LOVE·DOGS', 'Press Play', 'Andron', 'My Father, Die', 'Naradan', 'No No: A Dockumentary', 'Mosagallu', 'Kill Command', 'Thugs of Hindostan', 'Annabellum: The Curse of Salem', 'Inside the Dream', '20 Feet from Stardom', 'Love and Other Cults', 'Force of Nature', 'Hollower', 'Arjun Reddy', '31', 'Odd Couple', 'Init!ation', 'Sharknado 5: Global Swarming', 'After the Ball', 'My Days of Mercy', 'The Killing of Kenneth Chamberlain', 'Hairbrained', 'Obsession', 'Ayogya', '10', 'Meeting Mr. Christmas', 'Veerappan', 'Everything About Her', 'Yuddham Sharanam', 'The Alternate', 'The End of Meat', 'A Year in Champagne', 'Lost in London', 'Hatched', 'The Last Victims', "Forever's End", 'Christmas in the Heartland', 'Stiv', 'Perimortem', 'CODumentary', 'Tiger House', 'Jawaani Jaaneman', 'Redeeming Love', 'Land of Hope', 'Gimme Danger', 'Holy Hell', 'The Russian Bride', 'Summer Daydream', 'Anarchy Parlor', 'S.O.S. Mamis: La película', 'Laal Kaptaan', 'Buddymoon', 'Bobby Robson: More Than a Manager', 'Silencio', '47 Days', 'Jeem Boom Bhaa', 'Hushaaru', 'Welcome to Happiness', 'Joker', 'Velle', 'Chris Porter: A Man From Kansas', 'Sunny', 'Blue Mountain State: The Rise of Thadland', 'Ente Ummante Peru', 'Axone', "Something from Tiffany's", 'Momentum', 'Ophelia', 'The Resistance Fighter', 'Son of the South', 'Dum Laga Ke Haisha', 'Detective Chinatown', 'Barefoot', 'Control', 'Saina', 'Breakpoint: A Counter History of Progress', 'Untrapped: The Story of Lil Baby', 'Dark Signal', 'The Last Keepers', 'Fraud Saiyaan', 'Kotigobba 3', 'Sunshine Superman', 'Road to Hell', 'Kill Kane', 'Star', 'The Diabolical Schemes of Thadeus Jackson', 'Adventure Boyz', 'Chilling Visions: 5 Senses of Fear', 'Caged No More', 'Bastille ReOrchestrated', 'The Curse', 'Banjara: The Truck Driver', 'The Future of Work and Death', 'Margaret Cho: PsyCHO', 'Ikkat', 'Equals', 'Bliss', 'KevCoin: The Movie', 'Sacrifice', 'The Operative', 'Senior Moment', 'Maharshi', 'Alice Fades Away', 'Tribal Get Out Alive', 'Theeran Adhigaaram Ondru', 'Kaduva', 'The Dark Red', 'Mardaani 2', 'French Biriyani', 'Wall', 'Napoleon', 'Chhalaang', 'Touch Chesi Chudu', 'A Good American', 'FIR', 'The Scandalous Four', 'Satanic Panic', 'Boycie in Belgrade', 'Tomorrow Is Today', 'Once Upon A Holiday', "Mariah Carey's All I Want for Christmas Is You", 'Ek Mini Katha', 'My Son', 'Burn', 'The Report', 'Honeyspider', "We're All Fruit Salad!: The Wiggles' Greatest Hits", 'The Condemned 2', 'Love at First Child', 'Bull Shark', 'Vault', 'Patterns of Evidence: The Exodus', 'Break', 'The Ghazi Attack', 'The Unthinkable', 'V', 'Ghoster', 'Sharmaji Namkeen', 'The Great Father', 'Jalsa', 'Gurgaon', 'Hostile Border', 'Good Tidings', 'Casa Vita', 'Almost Mercy', 'Fight For Space', 'Witness Infection', 'Achmed Saves America', 'Greyson Family Christmas', 'Orange', 'The Sunset Six', 'Rogue', 'Broken Mile', 'Guardians', 'Brian Posehn: Posehn 25x2', 'A Alma de um Ciclista', 'Snoop Dogg: Uncaged', 'Clerks III', 'Horror Anthology Movie Volume 1', 'Santana IV - Live at The House of Blues, Las Vegas', 'The Bag Man', 'Anne of Green Gables', 'Ladyworld', 'Grace and Goliath', 'Whispers Behind the Wall', 'The Christmas Sitters', 'Becoming Cary Grant', 'Ishq', 'Yuvarathnaa', 'Sufiyum Sujatayum', 'Sivappu Manjal Pachai', '9 Ways to Hell', 'Lilith', 'And Then There Was Eve', '1986', 'Padi Padi Leche Manasu', 'Odu Raja Odu', 'Darkness Wakes', 'Home', 'Kanan Gill: Keep It Real', 'Honeytrap', 'Celeste', 'Sparks & Embers', 'Villager News: War', 'Dogged', 'The Last Secrets Of 9/11', 'A Seat at the Table', 'Sameer', 'New Money', 'Death of Me', 'The Wedding Year', 'Hagazussa', "Anna Karenina. Vronsky's Story", 'Never Let Go', 'Night of the Kings', 'Wolf Manor', 'The Big Lie: American Addict 2', 'Underdogs', 'Oh My Dog', "Ephraim's Rescue"]
all_movie_genres = []

# Iterate through the movie titles to get genres
for title in movie_titles:
    max_retries = 3
    retry_count = 0
    while retry_count < max_retries:
        try:
            genres = get_movie_genres(title)
            all_movie_genres.append({'Movie Title': title, 'Genres': genres})
            print(f"{title}:\nGenres: {genres}")
            break  # Exit the loop if successful
        except Exception as e:
            print(f"Error: {e}")
            retry_count += 1
            time.sleep(2)  # Wait for a few seconds before retrying

# Create a DataFrame with genres for all movies
df = pd.DataFrame(all_movie_genres)

# Display the DataFrame
print("\nDataFrame with Genres:")
print(df)


2023-12-27 10:12:58,734 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "genres"


Hours:
Genres: ['Action', 'Drama', 'History', 'Thriller', 'War']


2023-12-27 10:13:11,502 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "genres"


The Grinch:
Genres: ['Animation', 'Comedy', 'Family', 'Fantasy', 'Musical']


2023-12-27 10:13:19,542 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "genres"


...

Underdogs:
Genres: ['Animation', 'Adventure', 'Comedy', 'Family', 'Fantasy', 'Sport']


2023-12-27 15:27:37,758 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "genres"


Oh My Dog:
Genres: ['Drama', 'Family']


2023-12-27 15:27:42,840 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "genres"


Ephraim's Rescue:
Genres: ['Adventure', 'Drama', 'History']

DataFrame with Genres:
                         Movie Title  \
0                              Hours   
1                         The Grinch   
2                      In a World...   
3         All the Money in the World   
4                              Relic   
...                              ...   
1891                      Wolf Manor   
1892  The Big Lie: American Addict 2   
1893                       Underdogs   
1894                       Oh My Dog   
1895                Ephraim's Rescue   

                                                 Genres  
0               [Action, Drama, History, Thriller, War]  
1         [Animation, Comedy, Family, Fantasy, Musical]  
2                                              [Comedy]  
3          [Biography, Crime, Drama, Mystery, Thriller]  
4                    [Drama, Horror, Mystery, Thriller]  
...                                                 ...  
1891                         

In [41]:
df.to_csv('genres_amazon.csv', index = False)

In [43]:
df

,Movie Title,Genres
0,Hours,"[Action, Drama, History, Thriller, War]"
1,The Grinch,"[Animation, Comedy, Family, Fantasy, Musical]"
2,In a World...,[Comedy]
3,All the Money in the World,"[Biography, Crime, Drama, Mystery, Thriller]"
4,Relic,"[Drama, Horror, Mystery, Thriller]"
...,...,...
1891,Wolf Manor,[Horror]
1892,The Big Lie: American Addict 2,[Documentary]
1893,Underdogs,"[Animation, Adventure, Comedy, Family, Fantasy..."
1894,Oh My Dog,"[Drama, Family]"


In [44]:
import pandas as pd
from imdb import IMDb
from concurrent.futures import ThreadPoolExecutor
import time

def get_movie_certification_with_rating(movie_title):
    ia = IMDb()
    try:
        movie = ia.search_movie(movie_title)[0]
        ia.update(movie, info=['main', 'certificates'])

        certifications = movie.get('certificates')
        if certifications:
            for cert in certifications:
                if 'USA' in cert or 'UK' in cert or 'Canada' in cert:
                    return extract_rating_from_certification(cert)

            if certifications:
                return extract_rating_from_certification(certifications[0])
    except Exception as e:
        print(f"Error fetching data for {movie_title}: {e}")

    return f'Movie "{movie_title}" not found or certification with rating not found for USA, UK, or Canada'

def extract_rating_from_certification(certification):
    return certification.split('::')[0].split(':')[-1]

def process_movie(title):
    certification = get_movie_certification_with_rating(title)
    return {'Movie Title': title, 'Certification': certification}

# Example usage
movie_titles = ['Hours', 'The Grinch', 'In a World...', 'All the Money in the World']

# Use ThreadPoolExecutor for parallelization
with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_movie, movie_titles))

# Create a DataFrame from the list of results
df_certifications = pd.DataFrame(results)

# Display the DataFrame
print("\nDataFrame with Selected Certifications:")
print(df_certifications)


2023-12-27 19:55:31,069 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"
2023-12-27 19:55:41,302 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"
2023-12-27 19:55:42,934 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"
2023-12-27 19:55:49,887 ERROR [imdbpy] /Users/mac/opt/anaconda3/lib/python3.9/site-packages/imdb/__init__.py:844: unknown information set "certificates"



DataFrame with Selected Certifications:
                  Movie Title Certification
0                       Hours           14A
1                  The Grinch             G
2               In a World...           14A
3  All the Money in the World           14A
